# Testing data quality at scale with PyDeequ

Authors: Vitalina Komashko (komashk@), Calvin Wang (calviwan@), Chris Ghyzel (cghyzel@), Joan Aoanan (jaoanan@), Veronika Megler (meglerv@) 

__Updated June 2024 to use a new dataset, added additional library usage examples.__


This notebook accompanies AWS Blog post [Testing data quality at scale with PyDeequ](https://aws.amazon.com/blogs/big-data/testing-data-quality-at-scale-with-pydeequ/).

You generally write unit tests for your code, but do you also test your data? Incoming data quality can make or break your application. Incorrect, missing, or malformed data can have a large impact on production systems. Examples of data quality issues include the following:

- Missing values can lead to failures in the production system that require non-null values (`NullPointerException`)
- Changes in the distribution of data can lead to unexpected outputs of machine learning (ML) models
- Aggregations of incorrect data can lead to misguided business decisions

In this post, we introduce PyDeequ, an open source Python wrapper over [Deequ](https://aws.amazon.com/blogs/big-data/test-data-quality-at-scale-with-deequ/) (an open source tool developed and used at Amazon). Deequ is written in [Scala](https://www.scala-lang.org/), whereas PyDeequ allows you to use its data quality and testing capabilities from Python and PySpark, the language of choice for many data scientists. PyDeequ democratizes and extends the power of Deequ by allowing you to use it alongside the many data science libraries that are available in that language. Furthermore, PyDeequ allows for fluid interface with [pandas](https://pandas.pydata.org/) DataFrames as opposed to restricting within [Apache Spark](https://spark.apache.org/) DataFrames.

Deequ allows you to calculate data quality metrics for your dataset, define and verify data quality constraints, and be informed about changes in data distribution. Instead of implementing checks and verification algorithms on your own, you can focus on describing how your data should look. Deequ supports you by suggesting checks for you. Deequ is implemented on top of Apache Spark and is designed to scale with large datasets (billions of rows) that typically live in a data lake, distributed file system, or a data warehouse. PyDeequ gives you access to this capability, but also allows you to use it from the familiar environment of your Python [Jupyter](https://jupyter.org/) notebook.

## Deequ at Amazon 

Deequ is used internally at Amazon to verify the quality of many large production datasets. Dataset producers can add and edit data quality constraints. The system computes data quality metrics on a regular basis (with every new version of a dataset), verifies constraints defined by dataset producers, and publishes datasets to consumers in case of success. In error cases, dataset publication can be stopped, and producers are notified to take action. Data quality issues don’t propagate to consumer data pipelines, reducing their area of impact.

Deequ is also used within [Amazon SageMaker Model Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html#model-monitor-how-it-works). Now with the availability of PyDeequ, you can use it from a broader set of environments — [Amazon SageMaker](https://aws.amazon.com/sagemaker/), [AWS Glue](https://aws.amazon.com/glue/), [Amazon EMR](https://aws.amazon.com/emr/), and more.

## Overview of PyDeequ

Let’s look at PyDeequ’s main components, and how they relate to Deequ (shown in the following diagram). 

- __Metrics computation__ – Deequ computes data quality metrics, which are statistics such as completeness, maximum, or correlation. Deequ uses Spark to read from sources such as [Amazon Simple Storage Service](https://aws.amazon.com/s3/) (Amazon S3) and compute metrics through an optimized set of aggregation queries. You have direct access to the raw metrics computed on the data.
- __Constraint verification__ – As a user, you focus on defining a set of data quality constraints to be verified. Deequ takes care of deriving the required set of metrics to be computed on the data. Deequ generates a data quality report, which contains the result of the constraint verification.
- __Constraint suggestion__ – You can choose to define your own custom data quality constraints or use the automated constraint suggestion methods that profile the data to infer useful constraints.
- __Python wrappers__ – You can call each Deequ function using Python syntax. The wrappers translate the commands to the underlying Deequ calls and return their response.

![pydeequ-spark-components](../imgs/pydeequ_architecture.jpg)

**Figure 1. Overview of PyDeequ components.** 

## Solution overview 

As a running example, we have generated a synthetic reviews dataset and introduced various data issues. We demonstrate how to detect these issues using PyDeequ. We begin the way many data science projects do: with initial data exploration and assessment in a Jupyter notebook.

During the data exploration phase, we want to answer some basic questions about the data:

- Are there fields that have missing values?
- How many distinct categories are there in the categorical fields?
- Are there correlations between some key features?
- If there are two supposedly similar datasets (such as different categories or different time periods), are they really similar?
- We also show you how to scale this approach to large-scale datasets, using the same code on an EMR cluster. This is how you’d likely do your ML training as you move into a production setting.



## Setup

In this section we will show how to set up PyDeequ in [SageMaker Notebooks](https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html).

We use the default VPC for SageMaker Notebooks. The examples presented here use PyDeequ library version 1.2.0 (latest at the time of the update to this notebook) and tested in a SageMaker Notebook instance ml.m5.2xlarge, `conda_python3` kernel.
 
1. Create a new notebook instance. 

As of version 1.1.0, PyDeequ supports Spark up to version 3.3.0. Your PyDeequ version has to work with your version of Spark.

2. In the notebook, run the following lines in a code cell to specify `SPARK_VERSION`:

In [1]:
import os
os.environ["SPARK_VERSION"] = '3.3' 

3. Install PyDeequ module. For consistency, we'll set the PyDeequ version too.

In [5]:
!pip install pydeequ==1.2.0
!pip install sagemaker_pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 MB 22.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 25.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker_pyspark: filename=sagemaker_pyspark-1.4.5-py3-none-any.whl size=181610611 sha256=6a4aafb71a17028e620b800e74753489e22a0a85994b4457b38fd6228be9c6d7
  Stored in directory: /root/.cache/pip/wheels/04/b3/67/fc68544871b31eafddf693b74a2f944ede1ea4c87bb3f8da4b
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764020 sha256=14a118494d82f2867b489cc46ecd1b567915f8036dec1cc2ae619f1a7b48dd3b
  Stored in directory: /root/.cache/pip/wheels/05/75/73/81f84d174299abca38dd6a06a5b98b08ae25fce50ab8986fa1
Successfully built sagemaker_pyspark pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9
    Uninstalling py4j-0.10.9:
      Successfully uninstalled py4j-0

4. To import the modules, run the following commands in a code cell:

In [6]:
import sagemaker_pyspark
import pydeequ

This completes the steps specific to SageMaker Notebooks.

### Start a PySpark Session

In the cell below we import modules and set up a Spark session with the following configurations:

- `config("spark.driver.extraClassPath", classpath)` to prepend extra classpath entries to the classpath of the driver
- `config("spark.jars.packages", pydeequ.deequ_maven_coord)` to provide Maven of jars to include on the driver and executor classpaths
- `config("spark.jars.excludes", pydeequ.f2j_maven_coord` to exclude jars to avoid conflicts
- `config("spark.driver.memory", "15g")` to increase Java heap space
- `config("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")` to read the datetime values as is. In our synthetic dataset we introduced review years and dates such as 1696 to simulate a manual entry error. To ensure that these timestamps are read correctly, this configuration was necessary. See [Spark issue SPARK-31404](https://issues.apache.org/jira/browse/SPARK-31404) about the calendar switch in the version 3.0.

For a detailed explanation about these parameters, see [Spark Configuration](https://spark.apache.org/docs/latest/configuration.html).

In [7]:
from pyspark.sql import SparkSession, Row, DataFrame
import json
import pandas as pd

classpath = ":".join(sagemaker_pyspark.classpath_jars())

spark = (SparkSession
    .builder
    .config("spark.driver.extraClassPath", classpath)
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .config("spark.driver.memory", "15g")
    .config("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
    .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f27460f4-f4bb-40cf-8730-c9550629aaa3;1.0
	confs: [default]
	found com.amazon.deequ#deequ;2.0.3-spark-3.3 in central
	found org.scala-lang#scala-reflect;2.12.10 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sf.opencsv#opencsv;2.3 in central
	found com.github.rwl#jtransforms;2.4.0 in central
	found junit#junit;4.8.2 in central
	found org.apache.commons#commons-math3;3.2 in central
	found org.spire-math#spire_2.12;0.13.0 in central
	found org.spire-math#spire-macros_2.12;0.13.0 in central
	found org.typelevel#machinist_2.12;0.6.1 in central
	found com.chuusai#shapeless_2.12;2.3.2 in central
	found org.typelevel#macro-compat_2.12;1.1.1 in central
	fo

24/09/19 18:27:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Read the dataset 

In [13]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 24.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3


In [23]:
df = spark.read.csv("data/titanic.csv", header=True)

In [24]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

After you load the DataFrame, you can run `df.printSchema()` to view the schema of the dataset:

In [25]:
df.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



## Data Analysis 

Before we define checks on the data, we want to calculate some statistics for the dataset. As with Deequ, PyDeequ supports a rich set of metrics. For more information, see [Test data quality at scale with Deequ](https://aws.amazon.com/blogs/big-data/test-data-quality-at-scale-with-deequ/) or the [GitHub repo](https://github.com/awslabs/deequ/tree/master/src/main/scala/com/amazon/deequ/analyzers). In the following example, we use the [AnalysisRunner](https://github.com/awslabs/deequ/blob/master/src/main/scala/com/amazon/deequ/analyzers/runners/AnalysisRunner.scala) to capture the metrics we’re interested in:

In [27]:
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("PassengerId")) \
                    .addAnalyzer(Distinctness("PClass")) \
                    .addAnalyzer(Mean("Fare")) \
                    .run()
                    
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)

/usr/local/lib/python3.9/dist-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [28]:
pd.options.display.float_format = '{:,.7g}'.format
analysisResult_df.show()

+-------+-----------+------------+--------------------+
| entity|   instance|        name|               value|
+-------+-----------+------------+--------------------+
|Dataset|          *|        Size|               891.0|
| Column|PassengerId|Completeness|                 1.0|
| Column|     PClass|Distinctness|0.003367003367003367|
+-------+-----------+------------+--------------------+



From this, we learn the following:

- `review_id` has no missing values and approximately 99.27% of the values are distinct
- 74.99% of reviews have a `star_rating` of 4 or higher
- `total_votes` and `star_rating` are not correlated
- `helpful_votes` and `total_votes` are strongly correlated
- The average `star_rating` is 3.99
- The dataset contains 3,010,972 reviews

Sometimes, you may want to run multiple metrics on a single column. For example, you want to check that all reviews were written either after 1996 or before 2017. In this case, it’s helpful to provide a name for each metric in order to distinguish the results in the output:

In [29]:
analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Compliance("Fare", 
"Fare >= 21.000")) \
                    .run()
analysisResult_pd_df = AnalyzerContext.successMetricsAsDataFrame(spark,
analysisResult, pandas=True)
analysisResult_pd_df

,entity,instance,name,value
0,Column,Fare,Compliance,0.4118967


In [ ]:
analysisResult_json = AnalyzerContext.successMetricsAsJson(spark, analysisResult)
analysisResult_json

## Define and Run Tests for Data

After analyzing and understanding the data, we want to verify that the properties we have derived also hold for new versions of the dataset. By defining assertions on the data distribution as part of a data pipeline, we can make sure every processed dataset is of high quality, and that any application consuming the data can rely on it.

For writing tests on data, we start with the `VerificationSuite` and add [checks](https://github.com/awslabs/deequ/blob/master/src/main/scala/com/amazon/deequ/checks/Check.scala) on attributes of the data. In this example, we test for the following properties of our data:

- At least 3 million rows in total
- `review_id` is never null
- `review_id` is unique
- `star_rating` has a minimum of 1.0 and maximum of 5.0
- `marketplace` only contains `US`, `UK`, `DE`, `JP`, or `FR`
- `year` does not contain negative values
- `year` is between 1996 and 2017

The following code reflects the previous statements. For information about all available checks, see the [GitHub repo](https://github.com/awslabs/deequ/blob/master/src/main/scala/com/amazon/deequ/checks/Check.scala). You can run this directly in the Spark shell as previously explained:


In [35]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Titanic Survival")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasSize(lambda x: x >= 800) \
        .hasMin("Fare", lambda x: x > 10000) \
        .isComplete("Embarked")  \
        .isUnique("Passenger")  \
        .isComplete("marketplace")  \
        .isContainedIn("sex", ["male", "female"]) \
        .isNonNegative("age"))\
    .run()
    
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark,
checkResult, pandas=True)

Python Callback server started!


/usr/local/lib/python3.9/dist-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


Here we change the display settings of the DataFrame to ensure that the entire constraint message is visible.

In [37]:
checkResult_df

,check,check_level,check_status,constraint,constraint_status,constraint_message
0,Titanic Survival,Warning,Warning,SizeConstraint(Size(None)),Success,
1,Titanic Survival,Warning,Warning,"MinimumConstraint(Minimum(Fare,None))",Failure,Expected type of column Fare to be one of (Lon...
2,Titanic Survival,Warning,Warning,"CompletenessConstraint(Completeness(Embarked,N...",Failure,Value: 0.9977553310886644 does not meet the co...
3,Titanic Survival,Warning,Warning,UniquenessConstraint(Uniqueness(List(Passenger...,Failure,Input data does not include column Passenger!
4,Titanic Survival,Warning,Warning,CompletenessConstraint(Completeness(marketplac...,Failure,Input data does not include column marketplace!
5,Titanic Survival,Warning,Warning,ComplianceConstraint(Compliance(sex contained ...,Success,
6,Titanic Survival,Warning,Warning,ComplianceConstraint(Compliance(age is non-neg...,Success,


In [36]:
checkResult_df.style.set_properties(
    **{
        'overflow-wrap': 'break-word',
        'inline-size': '10px',
    }
)

,check,check_level,check_status,constraint,constraint_status,constraint_message
0,Titanic Survival,Warning,Warning,SizeConstraint(Size(None)),Success,
1,Titanic Survival,Warning,Warning,"MinimumConstraint(Minimum(Fare,None))",Failure,"Expected type of column Fare to be one of (LongType,IntegerType,DoubleType,org.apache.spark.sql.types.DecimalType$@45c276c2,ByteType,FloatType,ShortType), but found StringType instead!"
2,Titanic Survival,Warning,Warning,"CompletenessConstraint(Completeness(Embarked,None))",Failure,Value: 0.9977553310886644 does not meet the constraint requirement!
3,Titanic Survival,Warning,Warning,"UniquenessConstraint(Uniqueness(List(Passenger),None))",Failure,Input data does not include column Passenger!
4,Titanic Survival,Warning,Warning,"CompletenessConstraint(Completeness(marketplace,None))",Failure,Input data does not include column marketplace!
5,Titanic Survival,Warning,Warning,"ComplianceConstraint(Compliance(sex contained in male,female,`sex` IS NULL OR `sex` IN ('male','female'),None))",Success,
6,Titanic Survival,Warning,Warning,"ComplianceConstraint(Compliance(age is non-negative,COALESCE(CAST(age AS DECIMAL(20,10)), 0.0) >= 0,None))",Success,


After calling `run()`, PyDeequ translates your test description into Deequ, which translates it into a series of Spark jobs that are run to compute metrics on the data. Afterwards, it invokes your assertion functions (for example, `lambda x: x == 1.0` for the minimum star rating check) on these metrics to see if the constraints hold on the data. 

Interestingly, the `review_id` column isn’t unique, which resulted in a failure of the check on uniqueness. We can also look at all the metrics that Deequ computed for this check by running the following:

In [43]:
checkResults_df = VerificationResult.successMetricsAsDataFrame(spark, checkResult, pandas = True)
checkResults_df

/usr/local/lib/python3.9/dist-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


,entity,instance,name,value
0,Column,age is non-negative,Compliance,1
1,Column,"sex contained in male,female",Compliance,1
2,Dataset,*,Size,891
3,Column,Embarked,Completeness,0.9977553


## Automated Constraint Suggestion 

If you own a large number of datasets or if your dataset has many columns, it may be challenging for you to manually define appropriate constraints. Deequ can automatically suggest useful constraints based on the data distribution. Deequ first runs a data profiling method and then applies a set of rules on the result. For more information about how to run a data profiling method, see the [GitHub repo](https://github.com/awslabs/deequ/blob/master/src/main/scala/com/amazon/deequ/examples/data_profiling_example.md).

In [ ]:
from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(df) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions in JSON format
print(json.dumps(suggestionResult, indent=2))

The result contains a list of constraints with descriptions and Python code, so that you can directly apply it in your data quality checks. You can call `print(json.dumps(result_json))` to inspect the suggested constraints.

# Scaling to Production 

So far, we’ve shown you how to use these capabilities in the context of data exploration using a Jupyter notebook running on a SageMaker notebook instance. As your project matures, you need to use the same capabilities on larger and larger datasets, and in a production environment. With PyDeequ, it’s straightforward to make that transition. The following diagram illustrates deployment options for local and production purposes on AWS.

![pydeequ-in-production](../imgs/pydeequ_deployment.png)

**Figure 3. Deployment of PyDeequ in production.** 

As seen in the diagram above, you can leverage both an AWS EMR cluster and/or AWS Glue for larger or production purposes. To learn more about how to configure an EMR cluster with PyDeequ to explore much larger volumes of data please refer to the AWS blog post [Testing data quality at scale with PyDeequ](https://aws.amazon.com/blogs/big-data/testing-data-quality-at-scale-with-pydeequ/).

## More Examples on GitHub

You can find examples of more advanced features on the [Deequ GitHub repo](https://github.com/awslabs/deequ):

- Deequ provides more than data quality checks with fixed thresholds. Learn how to use [anomaly detection on data quality metrics](https://github.com/awslabs/deequ/blob/master/src/main/scala/com/amazon/deequ/examples/anomaly_detection_example.md) to apply tests on metrics that change over time.
- Deequ offers support for storing and loading metrics. Learn how to use the [MetricsRepository](https://github.com/awslabs/deequ/blob/master/src/main/scala/com/amazon/deequ/examples/metrics_repository_example.md) for this use case.
- If your dataset grows over time or is partitioned, you can use Deequ’s [incremental metrics computation](https://github.com/awslabs/deequ/blob/master/src/main/scala/com/amazon/deequ/examples/algebraic_states_example.md). For each partition, Deequ stores a state for each computed metric. To compute metrics for the union of partitions, Deequ can use these states to efficiently derive overall metrics without reloading the data.

## Conclusion

This notebook showed you how to use PyDeequ for calculating data quality metrics, verifying data quality metrics, and profiling data to automate the configuration of data quality checks in an Amazon SageMaker notebook. PyDeequ is available using `pip install` and on GitHub for you to build your own data quality management pipeline.

Learn more about the inner workings of Deequ in the VLDB 2018 paper [Automating large-scale data quality verification](https://www.vldb.org/pvldb/vol11/p1781-schelter.pdf).